<a href="https://colab.research.google.com/github/sowmyarshetty/Project2-AI-Class/blob/main/Project2_AI_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import gdown
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#Read the Electricvehiclepopulation dataset
df_evp = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/P2Resources/Electric_Vehicle_Population_Data_for_WA.csv')
df_evp.head(5)




,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WBY8P4C55K,King,Bothell,WA,98011.0,2019,BMW,I3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,126.0,0.0,1.0,249382416,POINT (-122.20563 47.76144),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
1,KNDCE3LG1K,Thurston,Olympia,WA,98502.0,2019,KIA,NIRO,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,239.0,0.0,22.0,102638791,POINT (-122.92333 47.03779),PUGET SOUND ENERGY INC,5.306701e+10
2,5YJYGDEE8L,King,Seattle,WA,98125.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291.0,0.0,46.0,112603774,POINT (-122.30253 47.72656),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
3,1N4AZ0CP0F,Thurston,Olympia,WA,98506.0,2015,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,84.0,0.0,22.0,187006893,POINT (-122.87741 47.05997),PUGET SOUND ENERGY INC,5.306701e+10
4,WA1E2AFY8R,King,Seattle,WA,98119.0,2024,AUDI,Q5 E,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,23.0,0.0,36.0,271736741,POINT (-122.3684 47.64586),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10


In [3]:
#Read the Charging Stations dataset
df_cs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/P2Resources/charging_stations.csv')
df_cs.head(5)


<ipython-input-3-af152cf9e093>:2: DtypeWarning: Columns (6,20,31,46,63,69,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df_cs = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/P2Resources/charging_stations.csv')


,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,RD Blends,RD Blends (French),RD Blended with Biodiesel,RD Maximum Biodiesel Level,NPS Unit Name,CNG Station Sells Renewable Natural Gas,LNG Station Sells Renewable Natural Gas,Maximum Vehicle Class,EV Workplace Charging,Funding Sources
0,ELEC,Los Angeles Convention Center,1201 S Figueroa St,West hall and South hall,Los Angeles,CA,90015,NaN,213-741-1151,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LD,False,NaN
1,ELEC,Scripps Green Hospital,10666 N Torrey Pines Rd,"Patient Parking Structure, level G",La Jolla,CA,92037,NaN,NaN,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,ELEC,Galpin Motors,15421 Roscoe Blvd,NaN,Sepulveda,CA,91343,NaN,800-256-6219,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MD,False,NaN
3,ELEC,Galleria at Tyler,1299 Galleria at Tyler,NaN,Riverside,CA,92503,NaN,951-351-3110,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LD,False,NaN
4,ELEC,City of Pasadena - Holly Street Garage,150 E Holly St,NaN,Pasadena,CA,91103,NaN,626-744-7665,E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LD,False,NaN


In [5]:
#Read the Charging Stations dataset
df_demographics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/P2Resources/Demographics.csv')
df_demographics.head(5)


,SEQUENCE,FILTER,COUNTY,JURISDICTION,POP_1990,POP_1991,POP_1992,POP_1993,POP_1994,POP_1995,...,POP_2015,POP_2016,POP_2017,POP_2018,POP_2019,POP_2020,POP_2021,POP_2022,POP_2023,POP_2024
0,1,1,Adams,Adams County,13603.0,13823.0,14063.0,14335.0,14679.0,15030.0,...,19451,19643,20068,20200,20335,20613,20900,21100,21200,21475
1,2,2,Adams,Unincorporated Adams County,6466.0,6698.0,6776.0,7009.0,7162.0,7303.0,...,9093,9205,9250,9301,9375,9472,9575,9575,9585,9628
2,3,3,Adams,Incorporated Adams County,7137.0,7125.0,7287.0,7326.0,7517.0,7727.0,...,10358,10438,10818,10899,10960,11141,11325,11525,11615,11847
3,4,4,Adams,Hatton,71.0,80.0,81.0,82.0,83.0,84.0,...,90,87,85,82,82,79,80,80,80,167
4,5,4,Adams,Lind,472.0,400.0,523.0,435.0,452.0,451.0,...,565,556,556,534,535,535,535,535,535,535
